In [41]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
# Use Pandas to load dataset from csv file
import pandas as pd

# Make sure we don't get any GPU errors
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [42]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [43]:
# Check the shape of the training and test data
print("Train Images Shape:", train_images.shape)
print("Test Images Shape:", test_images.shape)
print("Train Labels Shape:", train_labels.shape)
print("Test Labels Shape:", test_labels.shape)

Train Images Shape: (60000, 28, 28)
Test Images Shape: (10000, 28, 28)
Train Labels Shape: (60000,)
Test Labels Shape: (10000,)


In [44]:
train_images = train_images.reshape(-1,28*28).astype("float32") / 255.0
test_images = test_images.reshape(-1,28*28).astype("float32") / 255.0

In [45]:
train_images.shape

(60000, 784)

In [46]:
class Dense(layers.Layer):
    def __init__(self, units):
        super(Dense, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            name="w",
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            name="b", shape=(self.units,), initializer="random_normal", trainable=True,
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [47]:
class MyReLU(layers.Layer):
    def __init__(self):
        super(MyReLU, self).__init__()

    def call(self, x):
        return tf.math.maximum(x, 0)

In [48]:
class MyModel(keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__()
        self.dense1 = Dense(64)
        self.dense2 = Dense(num_classes)
        self.relu = MyReLU()


    def call(self, x):
        x = self.relu(self.dense1(x))
        return self.dense2(x)

In [49]:
model = MyModel()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

In [50]:
model.fit(train_images, train_labels, batch_size=32, epochs=5, verbose =2 )

Epoch 1/5
1875/1875 - 6s - loss: 0.3537 - accuracy: 0.9026 - 6s/epoch - 3ms/step
Epoch 2/5
1875/1875 - 3s - loss: 0.1689 - accuracy: 0.9516 - 3s/epoch - 2ms/step
Epoch 3/5
1875/1875 - 4s - loss: 0.1191 - accuracy: 0.9648 - 4s/epoch - 2ms/step
Epoch 4/5
1875/1875 - 3s - loss: 0.0935 - accuracy: 0.9722 - 3s/epoch - 2ms/step
Epoch 5/5
1875/1875 - 3s - loss: 0.0768 - accuracy: 0.9772 - 3s/epoch - 2ms/step


In [51]:
model.evaluate(test_images, test_labels, batch_size=32, verbose =2 )

313/313 - 1s - loss: 0.1010 - accuracy: 0.9671 - 589ms/epoch - 2ms/step


[0.10099352896213531, 0.9671000242233276]